In [1]:
import numpy as np
import ccpy

In [2]:
from ccpy.interfaces.gamess_tools import load_from_gamess

nfrozen = 2
system, H = load_from_gamess("f2-2Re.log",
                             "onebody.inp",
                             "twobody.inp",
                             nfrozen)

In [3]:
from ccpy.models.calculation import Calculation
from ccpy.drivers.driver import cc_driver

cc_ground_state_calc = Calculation(
    order=2,
    calculation_type="ccsd",
    convergence_tolerance=1.0e-08,
)

T, total_energy, is_converged = cc_driver(cc_ground_state_calc, system, H)

   --------------------------------------------------
   Calculation type =  CCSD
   Maximum iterations = 60
   Convergence tolerance = 1e-08
   Energy shift = 0.0
   DIIS size =  6
   RHF symmetry = False
   --------------------------------------------------

   CC calculation started at 2022-03-04 12:26:00 

   Energy of initial guess =         0.0000000000

                 Iter.         Residuum                 δE                  ΔE               Wall time
-----------------------------------------------------------------------------------------------------
                   0         0.7713827724        -0.5760306882        -0.5760306882          (0.0m 0.1s)
                   1         0.2113849398         0.0049097381        -0.5711209501          (0.0m 0.1s)
                   2         0.0735896832        -0.0237563881        -0.5948773382          (0.0m 0.1s)
                   3         0.0228918799         0.0075473413        -0.5873299969          (0.0m 0.1s)
            

In [4]:
from ccpy.hbar.hbar_ccsd import build_hbar_ccsd
hbar = build_hbar_ccsd(T, H)

In [5]:
from ccpy.drivers.driver import lcc_driver

left_cc_ground_state_calc = Calculation(
    order=2,
    calculation_type="left_ccsd",
    convergence_tolerance=1.0e-08,
)

L, _, is_converged = lcc_driver(left_cc_ground_state_calc, system, T, hbar, omega=0.0, L=None, R=None)

   --------------------------------------------------
   Calculation type =  LEFT_CCSD
   Maximum iterations = 60
   Convergence tolerance = 1e-08
   Energy shift = 0.0
   DIIS size =  6
   RHF symmetry = False
   --------------------------------------------------

   CC calculation started at 2022-03-04 12:26:00 

   Energy of initial guess =         0.0000000000

                 Iter.         Residuum                 δE                  ΔE               Wall time
-----------------------------------------------------------------------------------------------------
                   0         0.9191252311         1.2753667688         1.2753667688          (0.0m 0.1s)
                   1         0.2190788586        -0.9507238284         0.3246429404          (0.0m 0.1s)
                   2         0.0912629653        -0.1821158215         0.1425271189          (0.0m 0.1s)
                   3         0.0766073998        -0.0184115567         0.1241155622          (0.0m 0.1s)
       

In [9]:
calculation = Calculation(
    order=3,
    calculation_type="ccsdt",
    convergence_tolerance=1.0e-08,
)

In [10]:
T, total_energy, is_converged = cc_driver(calculation, system, H)

        _____    _____    __ __      __  __   
       /\ __/\  /\ __/\  /_/\__/\  /\  /\  /\ 
       ) )__\/  ) )__\/  ) ) ) ) ) \ \ \/ / / 
      / / /    / / /    /_/ /_/ /   \ \__/ /  
      \ \ \_   \ \ \_   \ \ \_\/     \__/ /   
       ) )__/\  ) )__/\  )_) )       / / /    
       \/___\/  \/___\/  \_\/        \/_/     
                                              

   Authors:
    Karthik Gururangan (gururang@msu.edu)
    J. Emiliano Deustua (edeustua@caltech.edu)
    Affiliated with the Piecuch Group at Michigan State University


   System Information:
   ----------------------------------------------------
     Number of correlated electrons = 14
     Number of correlated orbitals = 28
     Number of frozen orbitals = 2
     Number of alpha occupied orbitals = 7
     Number of alpha unoccupied orbitals = 21
     Number of beta occupied orbitals = 7
     Number of beta unoccupied orbitals = 21
     Charge = 0
     Point group = D2H
     Symmetry of reference = AG
     Spin m

In [7]:
L.a.ov

AttributeError: 'numpy.ndarray' object has no attribute 'ov'

In [3]:
from ccpy.models.operators import ClusterOperator

T = ClusterOperator(system, 2)

setattr(T, 'a', np.load('t1a.npy'))
setattr(T, 'b', np.load('t1b.npy'))
setattr(T, 'aa', np.load('t2a.npy'))
setattr(T, 'ab', np.load('t2b.npy'))
setattr(T, 'bb', np.load('t2c.npy'))

In [4]:
from ccpy.drivers.cc_energy import get_cc_energy

Ecc = get_cc_energy(T, H)

print(Ecc)

-0.5924662868564046


In [5]:
from ccpy.hbar.hbar_ccsd import get_ccsd_intermediates

In [6]:
Hbar = get_ccsd_intermediates(T, H)

In [7]:
attrs = ['oo', 'ov', 'vv', 'vo']
for attr in attrs:
    value = getattr(Hbar.a, attr)
    print(attr,'->',np.linalg.norm(value.flatten()))

oo -> 2.86437750315122
ov -> 0.01651305045227316
vv -> 16.918561038990326
vo -> 2.231725716183271e-09


In [8]:
attrs = ['oo', 'ov', 'vv', 'vo']
for attr in attrs:
    value = getattr(Hbar.b, attr)
    print(attr,'->',np.linalg.norm(value.flatten()))

oo -> 2.8643775031474696
ov -> 0.016513050455755147
vv -> 16.918561038992458
vo -> 2.231725716183271e-09


In [7]:
attrs = ['oooo', 'vvvv', 'vvov', 'vooo', 'voov', 'ooov', 'vovv']
for attr in attrs:
    value = getattr(Hbar.aa, attr)
    print(attr,'->',np.linalg.norm(value.flatten()))

oooo -> 4.385718144421259
vvvv -> 14.927953220478148
vvov -> 1.8449778953965024
vooo -> 0.9891864882945884
voov -> 6.470525588883125
ooov -> 1.488376171790874
vovv -> 2.724767524041812


In [8]:
attrs = ['oooo', 'vvvv', 'ovov', 'voov', 'ovvo', 'vovo',
         'ovoo', 'vooo', 'vvvo', 'vvov', 'ooov', 'vovv', 
         'ovvv', 'oovo']

for attr in attrs:
    value = getattr(Hbar.ab, attr)
    print(attr,'->',np.linalg.norm(value.flatten()))

oooo -> 4.255133986010844
vvvv -> 11.807623894375777
ovov -> 7.165131442343245
voov -> 1.2307709911989486
ovvo -> 1.2307709911999734
vovo -> 7.165131442332141
ovoo -> 1.259959751388678
vooo -> 1.259959751381858
vvvo -> 1.8904193357420627
vvov -> 1.8904193357374672
ooov -> 1.5420788723442092
vovv -> 2.4098503854845585
ovvv -> 2.4098503854867737
oovo -> 1.5420788723403278


In [9]:
attrs = ['oooo', 'vvvv', 'vvov', 'vooo', 'voov', 'ooov', 'vovv']
for attr in attrs:
    value = getattr(Hbar.bb, attr)
    print(attr,'->',np.linalg.norm(value.flatten()))

oooo -> 4.385718144411174
vvvv -> 14.92795322048783
vvov -> 1.8449778954059095
vooo -> 0.9891864882957732
voov -> 6.470525588877627
ooov -> 1.4883761717871005
vovv -> 2.7247675240435774
